In [19]:
# Load CIFAR-10 dataset
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [20]:
# Assume class 0 is the minority class
import numpy as np
minority_class_label = 0
minority_indices_train = np.where(y_train == minority_class_label)[0]
num_indices_to_keep=len(minority_indices_train) // 2
indices_to_keep_train = minority_indices_train[:num_indices_to_keep]
# Remove indices corresponding to minority class label from y_train
y_train_filtered = np.delete(y_train, indices_to_keep_train, axis=0)
x_train_filtered = np.delete(x_train, indices_to_keep_train, axis=0)

In [21]:
minority_indices_train = np.where(y_test == minority_class_label)[0]
num_indices_to_keep=len(minority_indices_train) // 2
indices_to_keep_train = minority_indices_train[:num_indices_to_keep]
# Remove indices corresponding to minority class label from y_train
y_train_filtered = np.delete(y_test, indices_to_keep_train, axis=0)
x_train_filtered = np.delete(x_test, indices_to_keep_train, axis=0)

array([   29,    30,    35, ..., 25070, 25082, 25083], dtype=int64)

In [47]:
import tensorflow as tf

# Define F1 score metric
def f1_score(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.argmax(y_pred, axis=-1), tf.float32)
    tp = tf.reduce_sum(y_true * y_pred)
    fp = tf.reduce_sum((1 - y_true) * y_pred)
    fn = tf.reduce_sum(y_true * (1 - y_pred))
    precision = tp / (tp + fp + tf.keras.backend.epsilon())
    recall = tp / (tp + fn + tf.keras.backend.epsilon())
    f1_score = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())
    return f1_score

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Preprocess input data
x_train = tf.keras.applications.mobilenet_v2.preprocess_input(x_train)
x_test = tf.keras.applications.mobilenet_v2.preprocess_input(x_test)

# Define the model architecture
inputs = tf.keras.Input(shape=(32, 32, 3))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(inputs)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Dropout(.5)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
outputs = tf.keras.layers.Dense(10, activation="softmax")(x)

# Create the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model with F1 score metric
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy", f1_score])

# Train the model
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.3865 - accuracy: 0.5115 - f1_score: 4.5475 - val_loss: 1.1717 - val_accuracy: 0.5900 - val_f1_score: 4.4512
Epoch 2/10
1563/1563 [==============================] - 12s 8ms/step - loss: 1.1077 - accuracy: 0.6094 - f1_score: 4.5235 - val_loss: 1.0542 - val_accuracy: 0.6326 - val_f1_score: 4.6218
Epoch 3/10
1563/1563 [==============================] - 10s 6ms/step - loss: 0.9884 - accuracy: 0.6517 - f1_score: 4.5235 - val_loss: 0.9961 - val_accuracy: 0.6466 - val_f1_score: 4.5817
Epoch 4/10
1563/1563 [==============================] - 10s 6ms/step - loss: 0.9032 - accuracy: 0.6797 - f1_score: 4.5138 - val_loss: 0.9655 - val_accuracy: 0.6626 - val_f1_score: 4.5414
Epoch 5/10
1563/1563 [==============================] - 9s 6ms/step - loss: 0.8366 - accuracy: 0.7048 - f1_score: 4.5075 - val_loss: 0.9659 - val_accuracy: 0.6592 - val_f1_score: 4.5554
Epoch 6/10
1563/1563 [==============================] - 10s 7ms/st

In [ ]:
v = history.history['val_f1_score']

47500

In [ ]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

def prelu1(x):
    return tf.where(x >= 0, x, 2.0*x)
def prelu(x):
    return tf.where(x >= 0, x, 0.5*x)

def create_model_with_parametric_relu(dropout_rate=0.5, alpha_values=[3.0, 1.0]):
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.Dense(1000, activation=prelu1)(x)
    x = tf.keras.layers.Dense(500, activation=prelu)(x)
    x = tf.keras.layers.Dense(100, activation=prelu)(x)
    outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

model = create_model_with_parametric_relu()
model.fit(x_train, y_train, epochs=10, batch_size=16, validation_data=(x_test, y_test))

Epoch 1/10
 678/3125 [=====>........................] - ETA: 19s - loss: 1.7773 - accuracy: 0.3811

In [55]:
import tensorflow as tf
from tensorflow.keras.models import Model
import numpy as np
layer_output = model.get_layer(index=8).output
prelu_output_model = Model(inputs=model.input, outputs=layer_output)
random_index = np.random.randint(0, len(x_test))
sample_image = x_test[random_index]
sample_image = np.expand_dims(sample_image, axis=0)
intermediate_output = prelu_output_model.predict(sample_image)
print("Intermediate representation after applying Parametric ReLU:")
print(intermediate_output)

Intermediate representation after applying Parametric ReLU:
[[-3.09623748e-01  3.65926415e-01 -3.30250442e-01  6.23789549e-01
  -1.08102870e+00  2.34093356e+00 -4.61948693e-01 -1.12589061e+00
  -1.30840349e+00 -2.56084251e+00 -1.05813372e+00  1.41008341e+00
  -4.20403183e-01 -7.92685077e-02 -1.38201416e+00 -5.53634405e-01
  -1.07329559e+00 -3.15363079e-01 -3.53681386e-01 -1.04790235e+00
  -5.67464173e-01 -4.41738188e-01 -2.16922855e+00 -1.16489100e+00
  -2.39455551e-01 -3.90322119e-01 -4.50135991e-02 -1.84005427e+00
   1.37504816e+00 -7.66315639e-01  3.26110870e-02 -3.77541333e-01
  -1.06517458e+00 -1.92681921e+00 -6.77433133e-01 -8.36783469e-01
  -5.76628387e-01 -3.24826717e-01 -1.06180763e+00 -6.30141139e-01
  -3.15058708e-01  7.93263912e-01 -6.07709765e-01 -2.50434786e-01
   1.18437004e+00  1.55070782e-01 -3.32778245e-01 -7.52352297e-01
  -3.01972222e+00 -3.53013843e-01 -7.62959838e-01 -2.95632005e-01
  -2.54063725e-01 -2.57728904e-01  2.26394534e-01 -1.31080019e+00
  -1.18543124e+0

In [56]:
np.mean(intermediate_output[0])

-0.32810825

In [2]:
import tensorflow as tf
def create_model(dropout_rate=0.5):
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.Dense(1000, activation="relu")(x)
    #x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.Dense(500, activation="relu")(x)
    #x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.Dense(100, activation="relu")(x)
    outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

m = create_model(0.5)
m.summary()

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 32, 32, 3)        0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 32, 32, 3)        0         
 a)                                                              
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                             